In [1]:
import gc
import time
import torch

import numpy as np
from tqdm.notebook import tqdm

import datasets
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM

In [2]:
device = "cpu"

if torch.backends.mps.is_built() and torch.backends.mps.is_available():
    device = "mps"

In [3]:
"""
candidate models 
- facebook/opt-350m
- distilgpt2
"""


model_id = "philschmid/gpt-j-6B-fp16-sharded"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model =  AutoModelForCausalLM.from_pretrained("./gpt-j-6B-fp16-sharded/", torch_dtype = torch.float16,local_files_only = True)



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
model = model.to(device)

In [5]:
text = "Write 5 sentences about India"
inputs = tokenizer(text, return_tensors="pt").to(device)

In [ ]:
%%time


generation_params = {
	"top_p" : 0.5,
	"temperature" : 1,
	"max_new_tokens" : 50,
	"min_new_tokens" : 10,
# 	"num_beams" : 3,
	"use_cache" : True,
	"no_repeat_ngram_size" : 4,
}

output = model.generate(**inputs, **generation_params)
tokenizer.batch_decode(output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [5]:
!pwd

/Users/praful932/myfiles/code/bhaasha/notebooks


In [121]:
from datasets import load_dataset

dataset = load_dataset("samsum")

ImportError: To be able to use samsum, you need to install the following dependency: py7zr.
Please install it using 'pip install py7zr' for instance.

In [81]:
def run_inference(
    model_input: str,
    tokenizer_encoding_kwargs,
    generation_kwargs,
    run_times: int = 10,
):
    all_latencies = []
    for _ in tqdm(range(run_times)):
        gc.collect()
        start = time.time()
        input_ids = tokenizer(
            text=model_input, **tokenizer_encoding_kwargs, return_tensors="pt"
        ).input_ids.to(device)
        output_ids = model.generate(input_ids, **generation_kwargs)[0]
        decoded_output = tokenizer.decode(
            token_ids=output_ids, skip_special_tokens=True
        )
        print(decoded_output)
        print("===" * 15)
        end = time.time()
        latency = (end - start) * 1000
        all_latencies.append(latency)
    print(f"Avg Latency {(end - start) * 1000} ms")
    return all_latencies

In [120]:
"""
we will use samsum validation & test set

we will find best params on the validation set

"""


model_input = "Emma: I’ve just fallen in love with this advent calendar! Awesome! I wanna one for my kids! Rob: I used to get one every year as a child! Loved them! Emma: Yeah, i remember! they were filled with chocolates! Lauren: they are different these days! much more sophisticated! Haha! Rob: yeah, they can be fabric/ wooden, shop bought/ homemade, filled with various stuff Emma: what do you fit inside? Lauren: small toys, Christmas decorations, creative stuff, hair bands & clips, stickers, pencils & rubbers, small puzzles, sweets Emma: WOW! That’s brill! X Lauren: i add one more very special thing as well- little notes asking my children to do something nice for someone else Rob: i like that! My sister adds notes asking her kids questions about christmas such as What did the 3 wise men bring? etc Lauren: i reckon it prepares them for Christmas Emma: and makes it more about traditions and being kind to other people Lauren: my children get very excited every time they get one! Emma: i can see why! :). Summarize : "
tokenizer_encoding_kwargs = {
    "max_length": 1000,
    "truncation": True,
    "padding": False,
}

generation_kwargs = {
    # 'temperature' : ,
    # 'num_beams' : 1,
    "no_repeat_ngram_size": 4,
    # 'min_new_tokens' : 1,
    # 'max_new_tokens' : 50,
}

all_latencies = run_inference(
    model_input=model_input,
    tokenizer_encoding_kwargs=tokenizer_encoding_kwargs,
    generation_kwargs=generation_kwargs,
    run_times=2,
)

  0%|          | 0/2 [00:00<?, ?it/s]

Emma is going to buy a Christmas advent calendar for her kids.
Emma is going to buy a Christmas advent calendar for her kids.
Avg Latency 621.2060451507568 ms


In [ ]:
def find_best